In [1]:
import pandas as pd
#Punctuations
import string
#reg Expressions
import re
#emojis with description
import demoji
#sentences to words
from nltk.tokenize import TreebankWordTokenizer
#paras to sentences
from nltk.tokenize import sent_tokenize
#stopwords
from nltk.corpus import stopwords
#for converting word to vector
import gensim.models.word2vec as w2v

In [2]:
file = pd.read_csv('IRAhandle_tweets_1.csv',nrows=1000)
#file.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,...,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN


In [3]:
# subset of entire csv based on (langugae = english and russian) as DATAFRAME

In [4]:
text = file[(file['language']=='English') | (file['language']=='Russian')]

In [5]:
text = text[['content','language']]

In [6]:
text_df = text.sort_values(['language'])
#text_df

In [7]:
text_list = text_df.values.tolist()
#text_list

In [8]:
ind_russian = []
ind_english = []

for i in range(len(text_list)):
    if text_list[i][1]=='Russian':
        ind_russian.append(i)
    else :
        ind_english.append(i)

In [9]:
#(1) filtering hyperlinks and 
#(2) remove numbers and convert text into lowercase
#(3) separate emojis from text. create a list of set of emojis, each set represents the emojis in its respective locations tweet 
#(4) identifying '#' and creating a list of hashtag words
#(5) sentence into words
#(6) filtering punctuations
#(7) stop-words for english and russian tweets

In [10]:
%%time
text_h = []
emoji_list = []
hashtags_list = []
punch = string.punctuation
stop_words_e = stopwords.words('english')
stop_words_r = stopwords.words('russian')

for i in range(len(text_list)):
    
    # (1)
    tweet = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', text_list[i][0], flags=re.MULTILINE)

    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    
    # (2)
    tweet = re.sub(r'\d+', '', tweet)
    tweet = tweet.lower()

    # (3)
    emoji_set = demoji.findall(tweet)
    emoji_list.append(emoji_set)
    
    # (4)
    hastags_per_tweet = re.findall('\#(\w*)',tweet)
    hashtags_list.append(hastags_per_tweet)
    
    # (5)
    tweet = TreebankWordTokenizer().tokenize(tweet)
    tweet.sort()
    
    # (6)
    tweet = [words for words in tweet if words not in punch]
    
    # (7)
    if i<len(ind_english):
        tweet = [words for words in tweet if words not in stop_words_e]
    else:
        tweet = [words for words in tweet if words not in stop_words_r]
    
    text_h.append(tweet)

Wall time: 590 ms


In [11]:
#text_h

In [12]:
# %%time
# #model = w2v.Word2Vec(size=100, window=20, min_count=200, workers=4) 
# Create CBOW model 
#model1 = w2v.Word2Vec(text_h, min_count = 1,size = 100, window = 5) 
# Create Skip Gram model 
#model2 = w2v.Word2Vec(text_h, min_count = 1, size = 100, window = 5, sg = 1) 

In [13]:
from gensim import corpora
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(text_h)

# convert tokenized documents into a document-term matrix
corpus=[dictionary.doc2bow(text) for text in text_h]

In [14]:
%%time
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.019*"trump" + 0.010*"причина" + 0.007*"realdonaldtrump" + 0.006*"flag"')
(1, '0.019*"trump" + 0.011*"president" + 0.009*"cnn" + 0.006*"mexico"')
(2, '0.015*"realdonaldtrump" + 0.013*"причина" + 0.012*"president" + 0.010*"“"')
(3, '0.008*"know" + 0.006*"dont" + 0.006*"trump" + 0.005*"happy"')
(4, '0.016*"trump" + 0.010*"president" + 0.010*"people" + 0.009*"realdonaldtrump"')
Wall time: 3.88 s


In [15]:
%%time
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\Mona\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


Wall time: 1min 45s


In [17]:
from gensim.models import CoherenceModel

# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=text_h, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.870451016183864

Coherence Score:  0.5757121463422921
